평당가격 이상치 제거필요
전용/거래금액에서 거르고 한번더 거를것인가??
같은데이터 다른가격..
상업지역별 특성. 건폐율 용적률 영향. https://brunch.co.kr/@hausplanner-cm/112
용도지역 https://brunch.co.kr/@hausplanner-cm/109
건축용도별 순위 수치화  https://m.blog.naver.com/PostView.nhn?blogId=kjcim2&logNo=221220910589&proxyReferer=https:%2F%2Fwww.google.com%2F
년월일 부동산추이

In [259]:
# import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# data read
df = pd.read_csv('dataset.csv', encoding ='cp949')  # 한글 Encoding 문제로 encoding = 'cp949'

# check
print('data 확인')
df.head()

data 확인


,시군구,유형,지번,도로명,용도지역,건축물주용도,도로조건,전용/연면적(㎡),대지면적(㎡),거래금액(만원),층,계약년월,계약일,건축년도
0,서울특별시강남구개포동,일반,1***,논현로,준주거,제2종근린생활,25m이상,381.42,367.3,"550,000",NaN,201708,23,1993.0
1,서울특별시강남구개포동,일반,1**,선릉로10길,준주거,제2종근린생활,8m미만,535.20,207.8,"390,000",NaN,201708,29,1987.0
2,서울특별시강남구개포동,집합,1*,개포로,일반상업,판매,25m이상,17.64,NaN,"39,500",1.0,201709,4,1997.0
3,서울특별시강남구개포동,집합,1**,개포로,일반상업,제2종근린생활,25m미만,82.70,NaN,"60,000",NaN,201709,14,1984.0
4,서울특별시강남구개포동,집합,1**,개포로,제3종일반주거,제1종근린생활,25m이상,9.00,NaN,"15,000",1.0,201709,27,1983.0


In [260]:
#################################################################################
# TODO: 데이터 타입 처리                                                        #
#                                                                               #
# 1. 계약 년월, 계약일을 datetime 형식으로 변환 (datetime.datetime 함수 사용)   #
# 2. 건축년도 nan 값 처리 및 datetime 형식으로 변환 (datetime.datetime 함수사용)#
# 3. 거래금액 string -> float 로 형변환 (Dataframe.apply 사용)                  #
# 4. 변수명은 계약년월_adj, 건축년도_adj, '거래금액(만원)_adj' 저장             #
#################################################################################

# 1. 계약 년월, 계약일을 datetime 형식으로 변환 (datetime.datetime 함수 사용)
import datetime
import math
contract_yr = []
for i, j in np.array(df[['계약년월', '계약일']]):
    contract_yr.append(datetime.datetime(int(str(int(i))[:4]), int(str(int(i))[4:6]), j))
df['계약년월_adj'] = contract_yr






# 2. 건축년도 nan 값 처리 및 datetime 형식으로 변환 (datetime.datetime 함수사용)
df['건축년도'] = df['건축년도'].replace(0,np.nan)
df['건축년도'] = df['건축년도'].replace('-',np.nan)
construct_yr =[]
for i in np.array(df['건축년도']):
    if(math.isnan(i)):
        construct_yr.append(None)
    elif(i==0):
        construct_yr.append(0)
    else:
        construct_yr.append(datetime.datetime(int(str(int(i))[:4]),1,1)) 

df['건축년도_adj']=construct_yr
df['건축년도_adj']=df['건축년도_adj'].apply(pd.to_datetime, errors = 'coerce')






# 3. 거래금액 string -> float 로 형변환 (Dataframe.apply 사용)                #
df['거래금액(만원)_adj']=df['거래금액(만원)'].str.replace(',', '').astype('float64')









# 4. 변수명은 계약년월_adj, 건축년도_adj, '거래금액(만원)_adj' 저장 
### check
print("@데이터 차원 확인")
print("(데이터 수, 변수) =",df.shape)
print('\n\n\n')

print("@null 값 확인")
print(df.isnull().sum())
print('\n\n\n')

print('@데이터 형태 확인')
print(df.dtypes)



@데이터 차원 확인
(데이터 수, 변수) = (46874, 17)




@null 값 확인
시군구                 0
유형                  0
지번                  0
도로명                 0
용도지역                0
건축물주용도              0
도로조건                0
전용/연면적(㎡)           0
대지면적(㎡)         37910
거래금액(만원)            0
층               27463
계약년월                0
계약일                 0
건축년도              842
계약년월_adj            0
건축년도_adj          842
거래금액(만원)_adj        0
dtype: int64




@데이터 형태 확인
시군구                     object
유형                      object
지번                      object
도로명                     object
용도지역                    object
건축물주용도                  object
도로조건                    object
전용/연면적(㎡)              float64
대지면적(㎡)                float64
거래금액(만원)                object
층                      float64
계약년월                     int64
계약일                      int64
건축년도                   float64
계약년월_adj        datetime64[ns]
건축년도_adj        datetime64[ns]
거래금액(만원)_adj           float64
dtype: object


In [262]:
#################################################################################
# TODO: 합리적인 방법으로 Null 값을 처리                                        #
#                                                                               #
# 1. '대지면적(㎡)'은 거의다 비어있으므로 사용하지 않고                         #
# 2. '층'과 '도로조건'은 비어있는 값들을 최빈값을 대체 (DataFrame.replace 사용) # 
# 3. 건축년도가 비어있는 행은 제거 (index.isin 사용)                            #
# 4. 주요변수 전부 헨들링 되었으므로 그 외 nan이 존재하는 행은 드랍    
# 5. 다른 방법도 주석과 함께 제시 해볼 것.                                      #
#################################################################################
df_1 = df.copy()

# 0. 도로명, 도로조건 변수의 (0,'-')값 결측치로 처리'
#check
print('@@@step0')
print("@null 처리 전")
print(df_1.isnull().sum())
print('\n\n')

df_1 = df_1.replace(0,np.nan)
df_1 = df_1.replace('-',np.nan)

#check
print("@null 처리 후")
print(df_1.isnull().sum())
print('\n\n\n')





# 1. '대지면적(㎡)' 열 제거
#check
print('@@@step1 대지면적(㎡) 열 제거')
print("처리 전 (데이터 수, 변수) =",df_1.shape)

df_1 = df_1.drop(['대지면적(㎡)'],axis=1)

#check
print("처리 후 (데이터 수, 변수) =",df_1.shape)
print('\n\n\n')




# 2. '층'과 '도로조건' 비어있는 값들을 최빈값 대체

from collections import Counter
cnt = Counter(df_1['층'])
cnt2 = Counter(df_1['도로조건'])

#check
print('@@@step2 층과 도로조건 비어있는 값들을 최빈값 대체')
print('층 최빈값 = ' , cnt.most_common()[:1],', 도로조건 최빈값 = ' , cnt2.most_common()[:1])
print('층 null수', df_1['층'].isnull().sum(),', 도로조건 null수', df_1['도로조건'].isnull().sum())
print('\n')

df_1 = df_1.replace({'층': np.nan}, {'층': cnt.most_common()[:1][0][0]})
df_1 = df_1.replace({'도로조건': np.nan}, {'도로조건': cnt2.most_common()[:1][0][0]})

#print('대체 ' , cnt.most_common()[:1][0][0])

#check
cnt = Counter(df_1['층'])
cnt2 = Counter(df_1['도로조건'])
print('층 최빈값 = ' , cnt.most_common()[:1],', 도로조건 최빈값 = ' , cnt2.most_common()[:1])
print('층 null수', df_1['층'].isnull().sum(),', 도로조건 null수', df_1['도로조건'].isnull().sum())
print('\n\n\n')


# 3. 건축년도가 비어있는 행은 제거 (index.isin 사용)                            #
#check
print('@@@step3 건축년도가 비어있는 행은 제거')
print('데이터 형태 =',df_1.shape,', null 개수 = ',df_1['건축년도'].isnull().sum())
print('\n\n\n')

df_1 = df_1[~df_1['건축년도'].isin([np.nan])]

#check
print('데이터 형태 =',df_1.shape,', null 개수 = ',df_1['건축년도'].isnull().sum())
print('\n')


#4 주요변수 전부 헨들링 되었으므로 그 외 nan이 존재하는 행은 드랍
#check
print('@@@step4 주요변수 전부 헨들링 되었으므로 그 외 nan이 존재하는 행은 드랍')
print('처리 전 =',df_1.shape)

df_1 = df_1.dropna(axis=0)

#check
print('처리 후 =',df_1.shape)

@@@step0
@null 처리 전
시군구                 0
유형                  0
지번                  0
도로명                 0
용도지역                0
건축물주용도              0
도로조건                0
전용/연면적(㎡)           0
대지면적(㎡)         37910
거래금액(만원)            0
층               27463
계약년월                0
계약일                 0
건축년도              842
계약년월_adj            0
건축년도_adj          842
거래금액(만원)_adj        0
dtype: int64



@null 처리 후
시군구                 0
유형                  0
지번                  0
도로명              1517
용도지역                0
건축물주용도              0
도로조건             1288
전용/연면적(㎡)           0
대지면적(㎡)         37910
거래금액(만원)            0
층               27463
계약년월                0
계약일                 0
건축년도              842
계약년월_adj            0
건축년도_adj          842
거래금액(만원)_adj        0
dtype: int64




@@@step1 대지면적(㎡) 열 제거
처리 전 (데이터 수, 변수) = (46874, 17)
처리 후 (데이터 수, 변수) = (46874, 16)




@@@step2 층과 도로조건 비어있는 값들을 최빈값 대체
층 최빈값 =  [(1.0, 5934)] , 도로조건 최빈값 =  [('25m이상', 22303)]
층 null수 274

In [209]:
### Column이 한글이면 에러가 발생할 수 있으므로 영문으로 사용하는 것을 추천 (아래 코드는 한글 폰트 사용)
### 각 plot 함수는 dataframe과 columns을 인풋으로 받아서 변수의 그림 출력
### plotHistogram은 histogram, plotPairplot은 컬럼별 scatter plot, plotBoxplot은 기준 컬럼(범주형 변수)에 대한 box plot
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='NanumBarunGothic')

def plotHistogram(_df, columns):
    if type(columns) != list :
        print("columns 타입이 리스트가 아닙니다.")
    
        return
    
    for _n, i in enumerate(columns):
        df.describe()
        fig=plt.figure(figsize=(20,8))
        fig.patch.set_facecolor('xkcd:white')

        plt.subplot(2,3,_n + 1)
        plt.title(i)
        df.loc[:,i].hist(bins=50)
    plt.show()
        
        
def plotPairplot(_df, columns):
    if type(columns) != list :
        print("columns 타입이 리스트가 아닙니다.")
    
        return
    sns.pairplot(_df.loc[:,columns])
    plt.show()

        
def plotBoxplot(_df, basis_col1, y_col2):
    _unique = _df.loc[:, basis_col1].unique()
    if len(_unique) < 50:
        print("기준 컬럼의 종류가 너무 많습니다.(50개 이상)")
    
    box_plot=[]
    count_ = []
    for n,i in enumerate(_unique):
        box_plot.append(list(_df[_df.loc[:, basis_col1]==i].loc[:, y_col2]))
        count_.append(len((list(_df[_df.loc[:, basis_col1]==i].loc[:, y_col2]))))

    print('Count   ',count_)
    plt.boxplot(box_plot, flierprops=dict(markerfacecolor='g', marker='D'));
    plt.xticks(range(1,len(_unique)+1),_unique)

In [ ]:
plotBoxplot(df_1, '층', '거래금액(만원)_adj')

In [ ]:
plotHistogram(df_1, ['전용/연면적(㎡)', '층', '계약일'])

In [ ]:
plotPairplot(df_1, ['거래금액(만원)_adj', '전용/연면적(㎡)', '층', '계약일'])

In [269]:
#################################################################################
# TODO: 합리적인 방법으로 이상치 값을 처리하고 산점도의 변화 확인               #
#                                                                               #
# 1. 1%의 이상치를 모두 제거, 층은 큰값 1%제거, 건축년도는 작은값 1% 제거       #
#  (np.where 사용)
# 2. 도로조건 '-' 제거                                                          #
#################################################################################
df_2 = df_1.copy()
print(df_2.shape)
# 층은 큰값 1% 제거, 건축년도는 작은값 1%
df_2.loc[:, '층'] = np.where(df_1.loc[:, '층'] > np.quantile(df_1.loc[:, '층'], 0.99), np.nan, df_1.loc[:,'층'])
df_2.loc[:, '건축년도'] = np.where(df_1.loc[:, '건축년도'] < np.quantile(df_1.loc[:, '건축년도'], 0.01), np.nan, df_1.loc[:,'건축년도'])
df_2.loc[:, '도로조건'] = df_1.loc[:, '도로조건'].replace('-', np.nan)
df_2 = df_2.dropna()

(44624, 16)


In [270]:
df_2.shape

(43869, 16)

In [216]:
df_2.dtypes

시군구                     object
유형                      object
지번                      object
도로명                     object
용도지역                    object
건축물주용도                  object
도로조건                    object
전용/연면적(㎡)              float64
거래금액(만원)                object
층                      float64
계약년월                     int64
계약일                      int64
건축년도                   float64
계약년월_adj        datetime64[ns]
건축년도_adj                object
거래금액(만원)_adj           float64
dtype: object

In [ ]:
#################################################################################
# TODO: 예측의 유의미한 영향을 미칠 수 있는 파생변수 생성                       #
# - 건물가격을 바로 예측할 수 있지만, 평당가격을 예측하는 모델을 만들어 전용면적에
# 곱하여 사용하도록 모델 설계. (DataFrame.apply 사용)                                                                                
# 1. 지역구 단위의 행정구역 새로운 변수 생성 (변수명 '지역구') '서울특별시XX구' or '서울특별시 XX구 ', '구로구' 등 예외처리
# 2. 건물 년식 일 단위로 하여 새로운 변수 생성 (변수명 '건물년식') 
# 3. 평당가격 변수 추가 
# 4. 그 외의 합리적인 새로운 파생변수 생성                                
#################################################################################
df_3 = df_2.copy()


# df_3를 사용하여 작성할 것

평당가격 아웃라이어제거

In [ ]:
CATEGORICAL = ['용도지역', '건축물주용도', '도로조건','지역구']
df_4 = df_3.loc[:,[ '전용/연면적(㎡)', '층', '건물년식', '평당가격', '거래금액(만원)_adj']]

In [ ]:
#################################################################################
# TODO:범주형 변수 처리                                                         #
#                                                                               #
# 범주형 변수를 Onehot encoding (Pandas.get_dummies 사용)                       #
# 변수의 종류당 하나의 dimension이 생기므로 차원이 매우 커짐. 따라서 해결할 수  #
# 있는 여러 존재하는데, 의미를 해석하여 ordinal하게 변수를 정의하거나 차원을    #
# 축소하는 MCA 알고리즘 등이 존재
# Ordinal 예시 : '용도지역'의 경우 평당 가격이 
# 상업지역 > ... > 준주거 > ... > 개발제한구역 이므로 이를 오름차순의 값으로 설정
#################################################################################






